In [47]:
def pascal_to_yolo(folder_path, class_file, destiny_folder):
    '''Converts a pascal format label to Yolo format
    Takes the labels from an xml file and stores it in a text file 
    Converts data from the format xyxy to normalized xywh
    The text file has each line as - class id, xn, yn, wn, hn
    
    Arguments : 
    1. folder_path : Path of folder containing xml files
    2. class_file : Text file containing classnames and class id
    3. destiny_folder : Folder to store the output text files.
    '''
    import xml.etree.ElementTree as ET
    import os
    import shutil
    
    if not os.path.exists(destiny_folder):
        os.makedirs(destiny_folder)

    for file in os.listdir(folder_path):
        if file.endswith('.jpg'):
            
            # Classnames
            classnames = []
            with open(class_file, 'r') as f:
                classnames = f.read().splitlines()
            
            # Declaring the txt file path
            txt_name = file.split('.jpg')[0]+'.txt'
            txt_path = os.path.join(destiny_folder, txt_name)
            
            # Parsing the xml files
            if file.split('.jpg')[0]+'.xml' in os.listdir(folder_path):
                tree = ET.parse(os.path.join(folder_path, file.split('.jpg')[0]+'.xml'))
                root = tree.getroot()
                img_w = int(root[4][0].text)
                img_h = int(root[4][1].text)
                
                for member in root.findall('object'):
                    class_name = member[0].text # class name
                        
                    # bbox coordinates
                    xmin = int(member[4][0].text)
                    ymin = int(member[4][1].text)
                    xmax = int(member[4][2].text)
                    ymax = int(member[4][3].text)
                    
                    w = xmax - xmin
                    w_norm = w/img_w
                    h = ymax - ymin
                    h_norm = h/img_h
                    
                    x_center = (xmax+xmin)/2
                    x_c_norm = x_center/img_w
                    y_center = (ymax+ymin)/2
                    y_c_norm = y_center/img_h
                    
                    class_index=classnames.index(class_name)
                    
                    # Bounding box attributes
                    bb_value = [str(class_index), str(x_c_norm), str(y_c_norm), str(w_norm), str(h_norm)]
                    
                    # writing to text file
                    with open(txt_path, 'a') as f:
                        f.write(','.join(bb_value))
                        f.write('\n')
                    
                f.close()
                
                #Copying the image also to destined folder
                shutil.copy(os.path.join(folder_path, file), os.path.join(destiny_folder, file))

In [48]:
pascal_to_yolo('ashi_data/', 'classes (1).txt', 'yolo_data')